In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('processeddataset/processed_paySim.csv')
df.head()

step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  hour  \
0  M1979787155             0.0             0.0        0               0     1   
1  M2044282225             0.0             0.0        0               0     1   
2   C553264065             0.0             0.0        1               0     1   
3    C38997010         21182.0             0.0        1               0     1   
4  M1230701703             0.0             0.0        0               0     1   

   day  is_weekend  balance_change_org  balance_change_dest  orig_txn_count  \
0    0           0            -9839.64                  0.0               1   
1    0           0            -1864.28                  0.0               1   
2    0           0             -181.00                  0.0               1   
3    0           0             -181.00             -21182.0               1   
4    0           0           -11668.14                  0.0               1   

   amount_to_balance_ratio  is_zero_balance_orig  is_zero_balance_dest  
0                 0.057834                     0                     1  
1                 0.087731                     0                     1  
2                 0.994505                     0                     1  
3                 0.994505                     0                     0  
4                 0.280788                     0                     1

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df['transaction_velocity_org'] = df.groupby('nameOrig')['step'].diff().fillna(0)
df['amount_velocity_org'] = df.groupby('nameOrig')['amount'].diff().fillna(0)

# Risk indicators from EDA insights
df['high_risk_hour'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)
df['high_risk_type'] = df['type'].isin(['TRANSFER', 'CASH_OUT']).astype(int)
df['large_amount_flag'] = (df['amount'] > df['amount'].quantile(0.95)).astype(int)

# Balance-based risk indicators
df['zero_balance_orig'] = (df['oldbalanceOrg'] == 0).astype(int)
df['zero_balance_dest'] = (df['oldbalanceDest'] == 0).astype(int)
df['balance_ratio_orig'] = df['amount'] / (df['oldbalanceOrg'] + 1)
df['balance_ratio_dest'] = df['amount'] / (df['oldbalanceDest'] + 1)

# Aggregated customer behavior
customer_features = df.groupby('nameOrig').agg({
    'amount': ['mean', 'std', 'count'],
    'step': ['min', 'max'],
    'isFraud': 'mean'
}).fillna(0)

customer_features.columns = ['cust_avg_amt', 'cust_std_amt', 'cust_txn_count', 
                            'cust_first_txn', 'cust_last_txn', 'cust_hist_fraud_rate']

df = df.merge(customer_features, left_on='nameOrig', right_index=True, how='left')
df.head()

step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  hour  \
0  M1979787155             0.0             0.0        0               0     1   
1  M2044282225             0.0             0.0        0               0     1   
2   C553264065             0.0             0.0        1               0     1   
3    C38997010         21182.0             0.0        1               0     1   
4  M1230701703             0.0             0.0        0               0     1   

   day  is_weekend  balance_change_org  balance_change_dest  orig_txn_count  \
0    0           0            -9839.64                  0.0               1   
1    0           0            -1864.28                  0.0               1   
2    0           0             -181.00                  0.0               1   
3    0           0             -181.00             -21182.0               1   
4    0           0           -11668.14                  0.0               1   

   amount_to_balance_ratio  is_zero_balance_orig  is_zero_balance_dest  \
0                 0.057834                     0                     1   
1                 0.087731                     0                     1   
2                 0.994505                     0                     1   
3                 0.994505                     0                     0   
4                 0.280788                     0                     1   

   transaction_velocity_org  amount_velocity_org  high_risk_hour  \
0                       0.0                  0.0               1   
1                       0.0                  0.0               1   
2                       0.0                  0.0               1   
3                       0.0                  0.0               1   
4                       0.0                  0.0               1   

   high_risk_type  large_amount_flag  zero_balance_orig  zero_balance_dest  \
0               0                  0                  0                  1   
1               0                  0                  0                  1   
2               1                  0                  0                  1   
3               1                  0                  0                  0   
4               0                  0                  0                  1   

   balance_ratio_orig  balance_ratio_dest  cust_avg_amt  cust_std_amt  \
0            0.057834         9839.640000       9839.64           0.0   
1            0.087731         1864.280000       1864.28           0.0   
2            0.994505          181.000000        181.00           0.0   
3            0.994505            0.008545        181.00           0.0   
4            0.280788        11668.140000      11668.14           0.0   

   cust_txn_count  cust_first_txn  cust_last_txn  cust_hist_fraud_rate  
0               1               1              1                   0.0  
1               1               1              1                   0.0  
2               1               1              1                   1.0  
3               1               1              1                   1.0  
4               1               1              1                   0.0

In [7]:
df.to_csv('processeddataset/processed_cust_features.csv', index=False)

In [5]:
df.head()

step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  hour  \
0  M1979787155             0.0             0.0        0               0     1   
1  M2044282225             0.0             0.0        0               0     1   
2   C553264065             0.0             0.0        1               0     1   
3    C38997010         21182.0             0.0        1               0     1   
4  M1230701703             0.0             0.0        0               0     1   

   day  is_weekend  balance_change_org  balance_change_dest  orig_txn_count  \
0    0           0            -9839.64                  0.0               1   
1    0           0            -1864.28                  0.0               1   
2    0           0             -181.00                  0.0               1   
3    0           0             -181.00             -21182.0               1   
4    0           0           -11668.14                  0.0               1   

   amount_to_balance_ratio  is_zero_balance_orig  is_zero_balance_dest  \
0                 0.057834                     0                     1   
1                 0.087731                     0                     1   
2                 0.994505                     0                     1   
3                 0.994505                     0                     0   
4                 0.280788                     0                     1   

   transaction_velocity_org  amount_velocity_org  high_risk_hour  \
0                       0.0                  0.0               1   
1                       0.0                  0.0               1   
2                       0.0                  0.0               1   
3                       0.0                  0.0               1   
4                       0.0                  0.0               1   

   high_risk_type  large_amount_flag  zero_balance_orig  zero_balance_dest  \
0               0                  0                  0                  1   
1               0                  0                  0                  1   
2               1                  0                  0                  1   
3               1                  0                  0                  0   
4               0                  0                  0                  1   

   balance_ratio_orig  balance_ratio_dest  cust_avg_amt  cust_std_amt  \
0            0.057834         9839.640000       9839.64           0.0   
1            0.087731         1864.280000       1864.28           0.0   
2            0.994505          181.000000        181.00           0.0   
3            0.994505            0.008545        181.00           0.0   
4            0.280788        11668.140000      11668.14           0.0   

   cust_txn_count  cust_first_txn  cust_last_txn  cust_hist_fraud_rate  
0               1               1              1                   0.0  
1               1               1              1                   0.0  
2               1               1              1                   1.0  
3               1               1              1                   1.0  
4               1               1              1                   0.0

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

In [28]:
import mlflow
import mlflow.sklearn

In [29]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [6]:
feature_columns = [
'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest',
'hour', 'is_weekend', 'high_risk_hour', 'high_risk_type', 'large_amount_flag',
'zero_balance_orig', 'zero_balance_dest', 'balance_ratio_orig', 'balance_ratio_dest',
'cust_avg_amt', 'cust_std_amt', 'cust_txn_count'
]
X = df[feature_columns]
y = df['isFraud']

In [7]:
new_df = df[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest',
'hour', 'is_weekend', 'high_risk_hour', 'high_risk_type', 'large_amount_flag',
'zero_balance_orig', 'zero_balance_dest', 'balance_ratio_orig', 'balance_ratio_dest',
'cust_avg_amt', 'cust_std_amt', 'cust_txn_count', 'isFraud']]
new_df.head()

amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  newbalanceDest  \
0   9839.64       170136.0       160296.36             0.0             0.0   
1   1864.28        21249.0        19384.72             0.0             0.0   
2    181.00          181.0            0.00             0.0             0.0   
3    181.00          181.0            0.00         21182.0             0.0   
4  11668.14        41554.0        29885.86             0.0             0.0   

   hour  is_weekend  high_risk_hour  high_risk_type  large_amount_flag  \
0     1           0               1               0                  0   
1     1           0               1               0                  0   
2     1           0               1               1                  0   
3     1           0               1               1                  0   
4     1           0               1               0                  0   

   zero_balance_orig  zero_balance_dest  balance_ratio_orig  \
0                  0                  1            0.057834   
1                  0                  1            0.087731   
2                  0                  1            0.994505   
3                  0                  0            0.994505   
4                  0                  1            0.280788   

   balance_ratio_dest  cust_avg_amt  cust_std_amt  cust_txn_count  isFraud  
0         9839.640000       9839.64           0.0               1        0  
1         1864.280000       1864.28           0.0               1        0  
2          181.000000        181.00           0.0               1        1  
3            0.008545        181.00           0.0               1        1  
4        11668.140000      11668.14           0.0               1        0

In [9]:
new_df['isFraud'].value_counts()

isFraud
0    6354407
1       8213
Name: count, dtype: int64

In [10]:
new_df.to_csv('processeddataset/final_feature_paySim.csv', index=False)

In [31]:
print("Original class distribution:", Counter(y))
print(f"Fraud rate: {y.mean():.4f} ({y.sum()} fraud cases out of {len(y)} total)")

Original class distribution: Counter({0: 6354407, 1: 8213})
Fraud rate: 0.0013 (8213 fraud cases out of 6362620 total)


In [ ]:
mlflow.set_experiment("Compare ROS vs RUS Strategies")

def evaluate_imbalance_strategies(X, y, feature_names):
        
    # Split the data - 
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"Training set distribution: {Counter(y_train)}")
    print(f"Test set distribution: {Counter(y_test)}")
    
    # Define strategies to compare
    strategies = {
        'no_sampling': {
            'sampler': None,
            'name': 'No Sampling (Baseline)'
        },
        'RandomOverSampler': {
            'sampler': RandomOverSampler(
                random_state=42,
                sampling_strategy='auto' # Can adjust to 'minority' or specific ratio
            ),
            'name': 'RandomOverSampler'
        },
        'random_undersampling': {
            'sampler': RandomUnderSampler(
                random_state=42,
                sampling_strategy='auto'  # Can adjust ratio
            ),
            'name': 'Random UnderSampling'
        }
    }
    
    results = {}
    
    for strategy_name, strategy_config in strategies.items():
        print(f"\n{'='*50}")
        print(f"Testing strategy: {strategy_config['name']}")
        print(f"{'='*50}")
        
        with mlflow.start_run(run_name=strategy_config['name']):
            # Create pipeline
            if strategy_config['sampler'] is None:
                pipeline = RandomForestClassifier(
                    n_estimators=100,
                    random_state=42,
                    class_weight='balanced'  # Use class weights when no sampling
                )
                X_resampled = X_train
                y_resampled = y_train
            else:
                pipeline = Pipeline([
                    ('sampler', strategy_config['sampler']),
                    ('classifier', RandomForestClassifier(
                        n_estimators=100,
                        random_state=42,
                        class_weight=None  # No need for class weights when sampling
                    ))
                ])
            
            # Log strategy parameters
            mlflow.log_param("strategy", strategy_config['name'])
            mlflow.log_param("classifier", "RandomForest")
            mlflow.log_param("training_samples_before", len(X_train))
            mlflow.log_param("training_samples_after", len(X_resampled))
            mlflow.log_param("fraud_cases_before", y_train.sum())
            mlflow.log_param("fraud_cases_after", y_resampled.sum())
            
            print(f"Training set size after sampling: {len(X_resampled)}")
            print(f"Class distribution after sampling: {Counter(y_resampled)}")
            
            # Train model
            pipeline.fit(X_resampled, y_resampled)
            
            # Make predictions
            if strategy_config['sampler'] is None:
                y_pred = pipeline.predict(X_test)
                y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            else:
                y_pred = pipeline.named_steps['classifier'].predict(X_test)
                y_pred_proba = pipeline.named_steps['classifier'].predict_proba(X_test)[:, 1]
            
            # Calculate metrics
            classification_rep = classification_report(y_test, y_pred, output_dict=True)
            cm = confusion_matrix(y_test, y_pred)
            roc_auc = roc_auc_score(y_test, y_pred_proba)
            
            # Extract key metrics
            precision_0 = classification_rep['0']['precision']
            recall_0 = classification_rep['0']['recall']
            f1_0 = classification_rep['0']['f1-score']
            
            precision_1 = classification_rep['1']['precision']
            recall_1 = classification_rep['1']['recall']
            f1_1 = classification_rep['1']['f1-score']
            
            accuracy = classification_rep['accuracy']
            
            # Business-oriented metrics
            tn, fp, fn, tp = cm.ravel()
            false_positive_rate = fp / (fp + tn)
            false_negative_rate = fn / (fn + tp)
            
            # Log metrics to MLflow
            mlflow.log_metric("test_accuracy", accuracy) # type: ignore
            mlflow.log_metric("test_roc_auc", roc_auc)
            mlflow.log_metric("fraud_precision", precision_1)
            mlflow.log_metric("fraud_recall", recall_1)
            mlflow.log_metric("fraud_f1", f1_1)
            mlflow.log_metric("non_fraud_precision", precision_0)
            mlflow.log_metric("non_fraud_recall", recall_0)
            mlflow.log_metric("non_fraud_f1", f1_0)
            mlflow.log_metric("false_positive_rate", false_positive_rate)
            mlflow.log_metric("false_negative_rate", false_negative_rate)
            mlflow.log_metric("true_positives", tp)
            mlflow.log_metric("false_positives", fp)
            mlflow.log_metric("true_negatives", tn)
            mlflow.log_metric("false_negatives", fn)
            
            # Log classification report as JSON
            mlflow.log_dict(classification_rep, "classification_report.json")
            
            # Log the model
            mlflow.sklearn.log_model(pipeline, f"model_{strategy_name}")
            
            # Store results
            results[strategy_name] = {
                'pipeline': pipeline,
                'predictions': y_pred,
                'probabilities': y_pred_proba,
                'classification_report': classification_rep,
                'confusion_matrix': cm,
                'roc_auc': roc_auc,
                'resampled_distribution': Counter(y_resampled),
                'false_positive_rate': false_positive_rate,
                'false_negative_rate': false_negative_rate
            }
            
            # Print results
            print(f"\nResults for {strategy_config['name']}:")
            print(f"ROC-AUC: {roc_auc:.4f}")
            print(f"Fraud Precision: {precision_1:.4f}")
            print(f"Fraud Recall: {recall_1:.4f}")
            print(f"Fraud F1-Score: {f1_1:.4f}")
            print(f"False Positive Rate: {false_positive_rate:.4f}")
            print(f"False Negative Rate: {false_negative_rate:.4f}")
            print(f"Confusion Matrix:\n{cm}")
    
    return results, X_test, y_test

In [37]:
results, X_test, y_test = evaluate_imbalance_strategies(X, y, feature_columns)

Training set distribution: Counter({0: 4448085, 1: 5749})
Test set distribution: Counter({0: 1906322, 1: 2464})

Testing strategy: No Sampling (Baseline)
Training set size after sampling: 4453834
Class distribution after sampling: Counter({0: 4448085, 1: 5749})


2025/10/19 15:51:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/19 15:52:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Results for No Sampling (Baseline):
ROC-AUC: 0.9984
Fraud Precision: 1.0000
Fraud Recall: 0.9963
Fraud F1-Score: 0.9982
False Positive Rate: 0.0000
False Negative Rate: 0.0037
Confusion Matrix:
[[1906322       0]
 [      9    2455]]
🏃 View run No Sampling (Baseline) at: http://127.0.0.1:5000/#/experiments/668251657838870715/runs/346cee05c70b4f36b843da027a414eed
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/668251657838870715

Testing strategy: RandomOverSampler
Training set size after sampling: 8896170
Class distribution after sampling: Counter({0: 4448085, 1: 4448085})


2025/10/19 16:36:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/19 16:37:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Results for RandomOverSampler:
ROC-AUC: 0.9984
Fraud Precision: 1.0000
Fraud Recall: 0.9963
Fraud F1-Score: 0.9982
False Positive Rate: 0.0000
False Negative Rate: 0.0037
Confusion Matrix:
[[1906322       0]
 [      9    2455]]
🏃 View run RandomOverSampler at: http://127.0.0.1:5000/#/experiments/668251657838870715/runs/b10a852e8e6443f2ab9ab25ef9359d8d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/668251657838870715

Testing strategy: Random UnderSampling
Training set size after sampling: 11498
Class distribution after sampling: Counter({0: 5749, 1: 5749})


2025/10/19 16:37:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/19 16:37:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Results for Random UnderSampling:
ROC-AUC: 0.9993
Fraud Precision: 0.7732
Fraud Recall: 0.9976
Fraud F1-Score: 0.8712
False Positive Rate: 0.0004
False Negative Rate: 0.0024
Confusion Matrix:
[[1905601     721]
 [      6    2458]]
🏃 View run Random UnderSampling at: http://127.0.0.1:5000/#/experiments/668251657838870715/runs/cf538ea8e59843acb67256224daaf3eb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/668251657838870715


In [40]:
def generate_comparison_report(results):
    """
    Generate a comprehensive comparison report of all strategies
    """
    comparison_data = []
    
    for strategy_name, result in results.items():
        cr = result['classification_report']
        comparison_data.append({
            'Strategy': strategy_name,
            'ROC-AUC': result['roc_auc'],
            'Accuracy': cr['accuracy'],
            'Fraud Precision': cr['1']['precision'],
            'Fraud Recall': cr['1']['recall'],
            'Fraud F1-Score': cr['1']['f1-score'],
            'Non-Fraud Precision': cr['0']['precision'],
            'Non-Fraud Recall': cr['0']['recall'],
            'False Positive Rate': result['false_positive_rate'],
            'False Negative Rate': result['false_negative_rate'],
            'Training Samples After': sum(result['resampled_distribution'].values())
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df = comparison_df.sort_values('Fraud F1-Score', ascending=False)
    
    # print("\n" + "="*80)
    # print("COMPREHENSIVE STRATEGY COMPARISON")
    # print("="*80)
    # print(comparison_df.round(4))
    
    return comparison_df

In [41]:
comparison_df = generate_comparison_report(results)
comparison_df.head()

Strategy   ROC-AUC  Accuracy  Fraud Precision  Fraud Recall  \
0           no_sampling  0.998376  0.999995         1.000000      0.996347   
1     RandomOverSampler  0.998376  0.999995         1.000000      0.996347   
2  random_undersampling  0.999296  0.999619         0.773199      0.997565   

   Fraud F1-Score  Non-Fraud Precision  Non-Fraud Recall  False Positive Rate  \
0        0.998170             0.999995          1.000000             0.000000   
1        0.998170             0.999995          1.000000             0.000000   
2        0.871168             0.999997          0.999622             0.000378   

   False Negative Rate  Training Samples After  
0             0.003653                 4453834  
1             0.003653                 8896170  
2             0.002435                   11498

In [47]:
comparison_df.to_csv('processeddataset/model_strategy_comparison.csv', index=False)

mlflow experiment tracking

- All three strategies achieve very high performance (above 0.999 in accuracy and 0.997 in fraud recall).

- ROS has the advantage of having the highest accuracy and perfect non_fraud_recall and precisions, but the fraud_recall is slightly lower than RUS.

- RUS has the highest fraud_recall (0.99770) but at the cost of slightly lower accuracy and non_fraud_recall.

- No Sampling has the second highest accuracy and the lowest fraud_recall.

In [63]:
def plot_strategy_comparison(comparison_df):
    """
    Create visual comparisons of different strategies
    """
    # plt.figure(figsize=(15, 8))
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.ravel()
    
    metrics_to_plot = ['Fraud Precision', 'Fraud Recall', 'Fraud F1-Score', 
                      'False Positive Rate', 'False Negative Rate', 'ROC-AUC']
    
    for i, metric in enumerate(metrics_to_plot, 0):
        ax = axes[i]
        # plt.subplot(3, 3, i)
        bars = ax.bar(comparison_df['Strategy'], comparison_df[metric])
        ax.set_title(f'{metric} by Strategy')
        ax.set_ylabel(metric)
        ax.tick_params(axis='x',rotation=45)
        
        # Add value labels on bars
        for bar, value in zip(bars, comparison_df[metric]):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f'{value:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('assets/imbalance_strategy_comparison.png', dpi=100, bbox_inches='tight')
    plt.show()

In [64]:
plot_strategy_comparison(comparison_df)

C:\Users\Ann\AppData\Local\Temp\ipykernel_22640\711824564.py:25: UserWarning: Tight layout not applied. tight_layout cannot make Axes height small enough to accommodate all Axes decorations.
  plt.tight_layout()
